In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
import pickle
from sentences import *
from DCS import *
from utilities import *
from IPython.display import display

In [3]:

goodFileDict = pickle.load(open('mergedGood_v5.p', 'rb'))

In [4]:
full_list_pf = os.listdir('../TextSegmentation/Pickle_Files/')
full_list_up = os.listdir('../TextSegmentation/Updated Pickles/')
full_list_corct = os.listdir('../TextSegmentation/corrected_10to20/')

In [ ]:
list(goodFileDict.keys())

In [5]:
df_m1_pf = pd.read_csv('extras/jigyasa@pf.csv', header=None, names=['file', 'cid', 'pos', 'lemma'])
df_m1_10to20 = pd.read_csv('extras/jigyasa@skt.csv', header=None, names=['file', 'cid', 'pos', 'lemma'])
df_m1_upd = pd.read_csv('extras/jigyasa@upd.csv', header=None, names=['file', 'cid', 'pos', 'lemma'])

df_m1_pf['path'] = df_m1_pf.apply(lambda row: '../TextSegmentation/Pickle_Files/%d.p' % row['file'], axis=1)
df_m1_10to20['path'] = df_m1_10to20.apply(lambda row: '../TextSegmentation/corrected_10to20/%d.p' % row['file'], axis=1)
df_m1_upd['path'] = df_m1_upd.apply(lambda row: '../TextSegmentation/Updated Pickles/%d.p' % row['file'], axis=1)

In [6]:
# ix = 27
# fx = df_m1_upd.loc[ix, 'file']
fx = 197287
display(df_m1_upd[df_m1_upd['file'] == fx])
fn = '%d.p' % fx
fp = '../TextSegmentation/Updated Pickles/' + fn
print(fn, fp)
skt, dcs = loadSentence(fn, fp)

,file,cid,pos,lemma,path


197287.p ../TextSegmentation/Updated Pickles/197287.p


NameError: name 'loadSentence' is not defined

In [ ]:
SeeSentence(skt)

In [ ]:
SeeDCS(dcs)

In [ ]:
print(df_m1_pf.shape)
# df_m1_upd = df_m1_upd[df_m1_upd['lemma'] != 'tad']
print(df_m1_pf.shape)
print(len(df_m1_pf['file'].unique()))

In [ ]:
# df_m1_pf = df_m1_pf[(df_m1_pf['lemma'] != 'tad') & (df_m1_pf['lemma'] != 'vE')]
# df_m1_10to20 = df_m1_10to20[(df_m1_10to20['lemma'] != 'tad') & (df_m1_10to20['lemma'] != 'vE')]
# df_m1_upd = df_m1_upd[(df_m1_upd['lemma'] != 'tad') & (df_m1_upd['lemma'] != 'vE')]

frames = [df_m1_pf, df_m1_10to20, df_m1_upd]
df_m1 = pd.concat(frames)
print(df_m1.shape[0])
print(len(df_m1['file'].unique()))

In [10]:
# ix = 27
# fx = df_m1_upd.loc[ix, 'file']
fx = 197287
display(df_m1[df_m1['file'] == fx])
fn = '%d.p' % fx
fp = '../TextSegmentation/Updated Pickles/' + fn
print(fn, fp)
skt, dcs = loadSentence(fn, fp)

,file,cid,pos,lemma,path
25464,197287,1,13,uparasa,../TextSegmentation/corrected_10to20/197287.p


197287.p ../TextSegmentation/Updated Pickles/197287.p


NameError: name 'loadSentence' is not defined

In [ ]:
s1 = set(df_m1['file'].unique())
s2 = set(goodFileDict)


s3 = s1.union(s2)
len(s3)

In [ ]:
df_bad = pd.DataFrame(df_m1['file'].value_counts())
problem = df_bad.loc[df_bad['file'] > 1, 'file'].index

In [ ]:
p = problem[6]
df_m1[df_m1['file'] == 370078]

In [ ]:
df_m1 = df_m1.drop_duplicates('file')



In [9]:
df_m1['fileN'] = df_m1.apply(lambda row: '%d.p' % row['file'] ,axis=1)
df_m1.head()

,file,cid,pos,lemma,path,fileN
0,60212,2,0,sPawika,../TextSegmentation/Pickle_Files/60212.p,60212.p
1,123581,6,6,ariMdama,../TextSegmentation/Pickle_Files/123581.p,123581.p
2,143245,0,11,sampad,../TextSegmentation/Pickle_Files/143245.p,143245.p
3,313674,2,0,vartay,../TextSegmentation/Pickle_Files/313674.p,313674.p
4,105436,5,0,antatas,../TextSegmentation/Pickle_Files/105436.p,105436.p


In [ ]:
missing1_Dict = {}
for index, row in df_m1.iterrows():
    missing1_Dict[row['fileN']] = (row['cid'], row['pos'], row['lemma'], row['path'])

In [ ]:
# pickle.dump(missing1_Dict, open('missing1Dict.p', 'wb'))

In [24]:
df_m1['status'] = 'rejected'
df_m1.head()

,file,cid,pos,lemma,path,fileN,status
0,60212,2,0,sPawika,../TextSegmentation/Pickle_Files/60212.p,60212.p,rejected
1,123581,6,6,ariMdama,../TextSegmentation/Pickle_Files/123581.p,123581.p,rejected
2,143245,0,11,sampad,../TextSegmentation/Pickle_Files/143245.p,143245.p,rejected
3,313674,2,0,vartay,../TextSegmentation/Pickle_Files/313674.p,313674.p,rejected
4,105436,5,0,antatas,../TextSegmentation/Pickle_Files/105436.p,105436.p,rejected


In [49]:
counter = 0
tCounter = 0
for fx in np.array(df_m1.loc[df_m1['status'] == 'rejected', 'file']):
    fn = '%d.p' % fx
    df_occur = df_m1[df_m1['file'] == fx]
    slemmas = {}   
    
    fp = df_occur.iloc[0].loc['path']
#     display(df_occur)
    try:
        skt, dcs = loadSentence(fn, fp)
    except IndexError:
        continue

    if(len(skt.chunk) != len(dcs.lemmas)):
        continue
    try:
        for index, row in df_occur.iterrows():
            ci = row['cid']
            pos = row['pos']
            ws = word_new(row['lemma'])
            ws.lemmas.append(row['lemma'])
            if pos in skt.chunk[ci].chunk_words:
                skt.chunk[ci].chunk_words[pos].append(ws)
            else:
                skt.chunk[ci].chunk_words[pos] = []
                skt.chunk[ci].chunk_words[pos].append(ws)
    except IndexError:
        display(df_occur)
        continue

    # NOW CHECK FULL COVERAGE
    goodFlag = FullCoverage(skt, dcs)
    if goodFlag:
        df_m1.loc[df_occur.index, 'status'] = 'accepted'
        counter += 1
    tCounter += 1
    if(tCounter % 200 == 0):
        print('Chekpoint', counter, 'of', tCounter)

,file,cid,pos,lemma,path,fileN,status
243,40320,2,9,vinirmuc,../TextSegmentation/Updated Pickles/40320.p,40320.p,rejected


In [51]:
df_m1.head()
df_m1.to_csv('extras/missing1Fixed[Accepted_v1].csv')

In [50]:
df_m1['status'].value_counts()

accepted    46183
rejected      173
Name: status, dtype: int64

In [52]:
s1 = set(df_m1.loc[df_m1['status'] == 'accepted','fileN'])
s2 = set(goodFileDict.keys())
s3 = s1.union(s2)
len(s3)

93753

In [42]:
s3

{'225977.p',
 '80727.p',
 '412094.p',
 '32786.p',
 '276960.p',
 '187268.p',
 '392022.p',
 '437211.p',
 '56375.p',
 '435452.p',
 '149920.p',
 '427775.p',
 '262215.p',
 '248095.p',
 '390431.p',
 '312783.p',
 '23300.p',
 '34931.p',
 '169093.p',
 '32804.p',
 '84229.p',
 '411870.p',
 '114313.p',
 '24681.p',
 '67937.p',
 '32411.p',
 '13480.p',
 '320621.p',
 '91892.p',
 '104.p',
 '8995.p',
 '32982.p',
 '247969.p',
 '283194.p',
 '375846.p',
 '440163.p',
 '407168.p',
 '30984.p',
 '276235.p',
 '319063.p',
 '16041.p',
 '281906.p',
 '35740.p',
 '332924.p',
 '293596.p',
 '14977.p',
 '351591.p',
 '41515.p',
 '157.p',
 '25789.p',
 '56637.p',
 '310413.p',
 '377270.p',
 '379667.p',
 '362437.p',
 '34020.p',
 '236034.p',
 '31059.p',
 '362384.p',
 '153667.p',
 '381995.p',
 '243034.p',
 '432753.p',
 '308500.p',
 '382543.p',
 '276658.p',
 '18286.p',
 '399364.p',
 '163110.p',
 '319146.p',
 '140544.p',
 '6374.p',
 '178304.p',
 '440125.p',
 '41376.p',
 '12210.p',
 '378589.p',
 '45326.p',
 '82831.p',
 '301772.p

In [53]:
len(s2)

49441

In [54]:
len(s1)

46183